In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !apt-get install p7zip-full
# !p7zip -d file_name.7z

In [ ]:
%cd /content/drive/MyDrive/Gp\ Dataset
# !sudo apt-get install p7zip-full
# !7z x gp.7z




/content/drive/MyDrive/Gp Dataset


In [ ]:
import pandas as pd

labeledData=pd.read_csv('labeled_data.csv')
#labeledData=labeledData.sample(frac=0.5,random_state=50,ignore_index=True)
#unlabeledData=pd.read_csv('unlabeled_data.csv')

In [ ]:
labeledData.head()

,title,text,publishing_date,source_num,label
0,المنتخب الوطني المغربي لأقل من 20 سنة يخوض تجم...,يخوض المنتخب الوطني المغربي لكرة القدم لأقل من...,2021-05-23T00:00:00,source_1,credible
1,وزير النقل اعمارة:السرعة عامل مسبب لحوادث السي...,ترأس عبد القادر اعمارة، وزير التجهيز و النقل و...,2021-05-22T00:00:00,source_1,credible
2,"ميسي يؤكد أن الفوز بكأس اسبانيا كان ""نقطة تحول""",أكد النجم الارجنتيني ليونيل ميسي أن فوز فريقه ...,2021-05-22T00:00:00,source_1,credible
3,دبلوماسي مغربي سابق بإسبانيا لـ2M.ma: خطأ مدري...,أكّد الدبلوماسي المغربي السابق بإسبانيا، عبد ا...,2021-05-22T00:00:00,source_1,credible
4,شركتان، فرنسية وبريطانية تعلنان عن نتائج إيجاب...,أعلنت شركتا سانوفي الفرنسية العملاقة في تصنيع ...,2021-05-17T00:00:00,source_1,credible


In [ ]:
# unlabeledData.head()

In [ ]:
import nltk
from nltk.corpus import stopwords
import unicodedata as ud
from nltk.stem.isri import ISRIStemmer
nltk.download('stopwords')
st = ISRIStemmer()
stopwords_list=stopwords.words('arabic')

def preprocess_text(text):
    t = ''.join(c for c in text if not ud.category(c).startswith('P'))
    textwords = nltk.word_tokenize(t)
    filteredsentences = []
    # print(textwords)
    for w in textwords:
        if w not in stopwords_list:
            filteredsentences.append(w)
    stem = []
    # print(filteredsentences)
    for l in filteredsentences:
        stem.append(st.stem(l))
    listToStr = ' '.join([str(elem) for elem in stem])
    return listToStr

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from tqdm import tqdm

nltk.download('punkt')
labeledData['label']=labeledData['label'].replace(['not credible','credible'],[0,1])
#print(samples['label'])
y=labeledData['label']
print(len(labeledData))
num=0
#print(samples)
for i in tqdm(range(len(labeledData))):
    #print(labeledData.loc[i,'text'])
    text = labeledData.loc[i,'title']
    #text=labeledData.loc[i,'text']
    published_date = labeledData.loc[i,'publishing_date']
    listToStr=preprocess_text(text)
    #print(len(listToStr))
    num+=len(listToStr)
    #print(stem)
    #print(listToStr)
    labeledData.loc[i,'preprocessed_text']=listToStr

    #list_string.append(listToStr)
    #labeledData[i,'preprocessed_str']=listToStr
    #print(stem)
    #print(len(list_string))
    #exit()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


374543


100%|██████████| 374543/374543 [03:14<00:00, 1926.20it/s]


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

def applytfidf(textdoc):
    vectorizer = TfidfVectorizer( max_df=0.8,min_df=5000)
    vectorizer.fit(textdoc)
    idf = vectorizer.idf_
    # print(dict(zip(vectorizer.get_feature_names_out(), idf)))
    tfidf_matrix = vectorizer.transform(textdoc)
    # n_components = 1000
    # svd = TruncatedSVD(n_components=n_components)
    # reduced_matrix = svd.fit_transform(tfidf_matrix)
    return tfidf_matrix#,reduced_matrix

In [ ]:
tfidf_matrix=0
print(tfidf_matrix)
avg=num/len(labeledData)
print("average="+str(avg))
textdoc=labeledData['preprocessed_text']
tfidf_matrix=applytfidf(textdoc)
#,reduced_matrix
#print(reduced_matrix.shape)
print(tfidf_matrix.get_shape())



num_features=tfidf_matrix.get_shape()[1]
#num_features=reduced_matrix.shape[1]

0
average=35.36197445954136
(374543, 107)


In [ ]:
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,SpatialDropout1D,Dropout,Conv1D,MaxPooling1D



def create_model1(numoffeatures):
    model=Sequential()
    model.add(Embedding(input_dim=numoffeatures,output_dim=50))
    model.add(LSTM(units=64))
    model.add(Dense(1,activation='sigmoid'))
    model.summary()
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model

def create_model2(numoffeatures):
    model_lstm = Sequential()
    model_lstm.add(Embedding(input_dim=numoffeatures, output_dim=256,))
    model_lstm.add(SpatialDropout1D(0.3))
    model_lstm.add(LSTM(256, dropout=0.3, recurrent_dropout=0.3))
    model_lstm.add(Dense(256, activation='relu'))
    model_lstm.add(Dropout(0.3))
    model_lstm.add(Dense(1, activation='sigmoid'))
    model_lstm.compile(
        loss='binary_crossentropy',
        optimizer='Adam',
        metrics=['accuracy']
    )
    model_lstm.summary()
    return model_lstm

def create_model3(numoffeatures):
    model_lstm=Sequential()
    model_lstm.add(Embedding(numoffeatures,100))
    model_lstm.add(Dropout(0.2))
    model_lstm.add(Conv1D(64, 5, activation='relu'))
    model_lstm.add(MaxPooling1D(pool_size=4))
    model_lstm.add(LSTM(20, return_sequences=True))
    model_lstm.add(LSTM(20))
    model_lstm.add(Dropout(0.2))
    model_lstm.add(Dense(512))
    model_lstm.add(Dropout(0.3))
    model_lstm.add(Dense(256))
    model_lstm.add(Dense(1, activation='sigmoid'))
    model_lstm.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    model_lstm.summary()
    return model_lstm



In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
num_features=tfidf_matrix.shape[1]




#model=training_model.create_model1(num_features)
model=create_model3(num_features)
tfidf_int=tfidf_matrix.toarray().astype(np.float32)
#reduced_matrix_int=reduced_matrix.toarray().astype('int32')
#print(len(tfidf_int))
#model.fit(tfidf_int,ysamples,epochs=5,verbose=1)
X_train,X_test,Y_train,Y_test=train_test_split(tfidf_int,y,test_size=0.2,random_state=20)


model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=5,verbose=1,batch_size=128)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, None, 100)         10700     
                                                                 
 dropout_9 (Dropout)         (None, None, 100)         0         
                                                                 
 conv1d_2 (Conv1D)           (None, None, 64)          32064     
                                                                 
 max_pooling1d_2 (MaxPooling  (None, None, 64)         0         
 1D)                                                             
                                                                 
 lstm_7 (LSTM)               (None, None, 20)          6800      
                                                                 
 lstm_8 (LSTM)               (None, 20)                3280      
                                                      